### 1. Imports

In [ ]:
# Imports
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import torch
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
import math
import time
import numpy as np
import sys
import argparse
import os
import matplotlib.pyplot as plt
import pandas as pd
import math

### 2. Data Pre-Processing

#### 2.1 Helper Functions

In [ ]:
# Encoding and decoding
def decode(vocab,corpus):
    
    text = ''
    for i in range(len(corpus)):
        wID = corpus[i]
        text = text + vocab[wID] + ' '
    return(text)

def encode(words,text):
    corpus = []
    tokens = text.split(' ')
    for t in tokens:
        try:
            wID = words[t][0]
        except:
            wID = words['<unk>'][0]
        corpus.append(wID)
    return(corpus)

def read_encode(file_name,vocab,words,corpus,threshold):
    
    wID = len(vocab)
    
    if threshold > -1:
        with open(file_name,'rt', encoding='utf8') as f:
            for line in f:
                line = line.replace('\n','')
                tokens = line.split(' ')
                for t in tokens:
                    try:
                        elem = words[t]
                    except:
                        elem = [wID,0]
                        vocab.append(t)
                        wID = wID + 1
                    elem[1] = elem[1] + 1
                    words[t] = elem

        temp = words
        words = {}
        vocab = []
        wID = 0
        words['<unk>'] = [wID,100]
        vocab.append('<unk>')
        for t in temp:
            if temp[t][1] >= threshold:
                vocab.append(t)
                wID = wID + 1
                words[t] = [wID,temp[t][1]]
            
                    
    with open(file_name,'rt', encoding='utf8') as f:
        for line in f:
            line = line.replace('\n','')
            tokens = line.split(' ')
            for t in tokens:
                try:
                    wID = words[t][0]
                except:
                    wID = words['<unk>'][0]
                corpus.append(wID)
                
    return [vocab,words,corpus]

def plot_data(x, y1, y2, xlabel, ylabel):
    plt.plot(x, y1, 'b')
    plt.plot(x, y2, 'r')
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.show()

### 3. Model

In [ ]:
# FeedForward Model
class LanguageFFNN(nn.Module):
    def __init__(self, vocab, words, d_model, d_hidden, dropout, ngram_size):
        super().__init__() 
    
        # Class parameters
        self.vocab = vocab
        self.words = words
        self.vocab_size = len(self.vocab)
        self.d_model = d_model
        self.d_hidden = d_hidden
        self.ngram_size = ngram_size
        
        # Dropout
        self.dropout = nn.Dropout(p=dropout)

        # Embedding Layers
        self.input_embedding = nn.Embedding(self.vocab_size, self.d_model)
        # self.output_embedding =  nn.Embedding(self.vocab_size, self.d_model)
        
        # self.embeds = nn.Embedding(self.vocab_size, self.d_model)

        # Linear Layers
        self.fc1 = nn.Linear(ngram_size * d_model, self.d_model)
        # self.fc2 = nn.Linear(d_hidden, self.vocab_size)
        self.output_embedding = nn.Linear(self.d_model, self.vocab_size)

        # Nonlinear Layer
        self.activation = nn.Tanh()

        # Setting weights
        self.init_weights()
                
    # Initialize weights for foward layer
    def init_weights(self):
        weight_range = 0.1
        # tie embedding layers
        self.output_embedding.weight.data = self.input_embedding.weight.data
        
        self.input_embedding.weight.data.uniform_(-weight_range, weight_range)
        self.fc1.weight.data.uniform_(-weight_range, weight_range)
        self.fc1.bias.data.zero_()


    # Forward
    def forward(self, src):
        # Embeddings are fed into the forward layer
        embeds = self.input_embedding(src).view(-1, self.d_model * self.ngram_size)
        x = self.dropout(self.activation(self.fc1(embeds)))
        x = self.output_embedding(x).view(-1, self.vocab_size)
        x = torch.softmax(x, dim=1)
        return x

### 4. Train

### 5. Running Code


#### Loading Parameters

In [ ]:
class Params:
    def __init__(self, **kwargs):
        for key, value in kwargs.items():
            setattr(self, key, value)
            
model_map = {0: 'FFNN', 1: 'LSTM', 2: 'FFNN_CLASSIFY', 3: 'LSTM_CLASSIFY'}
train_map = {0: 'data/real.train.tok', 1: 'data/fake.train.tok', 2: 'data/mix.train.tok'}
valid_map = {0: 'data/real.valid.tok', 1: 'data/fake.valid.tok', 2: 'data/mix.valid.tok'}
test_map = {0: 'data/real.test.tok', 1: 'data/fake.test.tok', 2: 'data/mix.test.tok', 3: 'data/blind.test.tok'}

model_type = model_map[2]

# Types of data
train_type = train_map[2]
valid_type = valid_map[2]
test_type = test_map[2]

args = {
    "d_model": 4,
    "d_hidden": 4,
    "n_layers": 3,
    "batch_size": 64,
    "seq_len": 30,
    "printevery": 5000,
    "window": 3,
    "epochs": 1,
    "lr": 0.0001,
    "dropout": 0.35,
    "clip": 2.0,
    "model": model_type,
    "savename": model_type.lower(),
    "loadname": model_type.lower(),
    "trainname": train_type,
    "validname": valid_type,
    "testname": test_type
}

#### Main

In [ ]:
# Main Function
def main(args): 
    torch.manual_seed(0)
    
    params = Params(**args)
    train_name = params.trainname
    valid_name = params.validname
    test_name = params.testname
    model_type = params.model
    d_mod = params.d_model
    d_hid = params.d_hidden
    dropout = params.dropout
    epochs = params.epochs
    window = params.window
    batch_size = params.batch_size
    lr = params.lr

    [train_vocab,train_words,train] = read_encode(train_name,[],{},[],3)
    train_data = torch.tensor(train)

    [valid_vocab,valid_words,valid] = read_encode(valid_name,[],{},[],3)
    valid_data = torch.tensor(valid)

    [test_vocab,test_words,test] = read_encode(test_name,[],{},[],3)
    test_data = torch.tensor(test)
    
    if model_type == 'FFNN':
        pass
    if model_type == 'LSTM':
        pass
    if model_type == 'FFNN_CLASSIFY':
        pass
    if model_type == 'LSTM_CLASSIFY':
        pass


In [ ]:
main(args)